In [1]:

list_of_packages <- c("ggplot2", "dplyr", "DESeq2","biomaRt","tidyr")
for(package in list_of_packages){
suppressPackageStartupMessages(suppressMessages(suppressWarnings(library(package,character.only=TRUE))))
}

In [2]:
genes <- c(
 "GSDMD",
 "GSDME"
)

In [3]:
all_gtex_tissue_data <- file.path("../data/genes_of_interest_gtex_tissue_data.tsv")
gtex_df <- read.table(all_gtex_tissue_data, header = TRUE, sep = "\t")
# trim the decimals in Ensembl
gtex_df$Ensembl <- gsub("\\..*", "", gtex_df$Ensembl)
head(gtex_df)

,Ensembl,geneID,Sample,expression,tissue
,<chr>,<chr>,<chr>,<int>,<chr>
1,ENSG00000132906,CASP9,GTEX.1117F.0226.SM.5GZZ7,362,adipose_subcutaneous
2,ENSG00000132906,CASP9,GTEX.111CU.1826.SM.5GZYN,539,adipose_subcutaneous
3,ENSG00000132906,CASP9,GTEX.111FC.0226.SM.5N9B8,1071,adipose_subcutaneous
4,ENSG00000132906,CASP9,GTEX.111VG.2326.SM.5N9BK,1409,adipose_subcutaneous
5,ENSG00000132906,CASP9,GTEX.111YS.2426.SM.5GZZQ,567,adipose_subcutaneous
6,ENSG00000132906,CASP9,GTEX.1122O.2026.SM.9YFMG,1403,adipose_subcutaneous


In [4]:
# convert the GeneID to gene symbol
mart <- useMart(biomart="ENSEMBL_MART_ENSEMBL", dataset="hsapiens_gene_ensembl")
gene_id <- gtex_df$Ensembl
lookup <- getBM(
    mart = mart,
    attributes = c('hgnc_symbol','transcript_length','ensembl_gene_id'),
    filter = 'ensembl_gene_id',
    values = gene_id,
    uniqueRows = TRUE
    )
lookup
# merge the two datasets
# tmp_df <- merge(tmp_df, lookup, by.x="Description", by.y="hgnc_symbol", all=TRUE)

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Operation was aborted by an application callback


In [ ]:

# convert the GeneID to gene symbol
mart <- useMart(biomart="ENSEMBL_MART_ENSEMBL", dataset="hsapiens_gene_ensembl")
gene_id <- gtex_df$Description
lookup <- getBM(
    mart = mart,
    attributes = c('hgnc_symbol','transcript_length'),
    filter = 'hgnc_symbol',
    values = gene_id,
    uniqueRows = TRUE)
# merge the two datasets
gtex_df <- merge(gtex_df, lookup, by.x="Description", by.y="hgnc_symbol", all=TRUE)
# write the merged dataset to a file
head(gtex_df)

In [ ]:

gsdm_df <- gtex_df[gtex_df$Description %in% genes,]
head(gsdm_df)

In [ ]:
unique(gsdm_df$tissue)

In [ ]:
# get the mean expression of each gene in each tissue across all samples
gsdm_df <- gsdm_df %>% 
  group_by(Description, tissue) %>% 
  summarise(mean_expression = mean(expression), sd_expression = sd(expression)) %>%
  ungroup()
head(gtex_df)

In [ ]:
width <- 30
height <- 30
options(repr.plot.width = width, repr.plot.height = height)
gsdm_plot <- (
    ggplot(gsdm_df, aes(x = Description, y = mean_expression, fill = tissue))
    + geom_bar(stat = "identity", position = "dodge")
    + facet_wrap(~tissue, scales = "free_y", nrow = 10)
    + theme(legend.position = "none")
    + theme(strip.text = element_text(size = 18))

)
gsdm_plot

In [ ]:
tlr_and_casp_genes <- c(
 "TLR4",
 "TLR5",
 "TLR10",
 "CASP1",
 "CASP2",
 "CASP3",
 "CASP4",
 "CASP5",
 "CASP6",
 "CASP7",
 "CASP8",
 "CASP9",
 "CASP10",
 "CASP11"
)
casp_df <- gtex_df[gtex_df$Description %in% tlr_and_casp_genes,]
head(casp_df)

In [ ]:
# get the mean expression of each gene in each tissue across all samples
casp_df <- casp_df %>% 
  group_by(Description, tissue) %>% 
  summarise(mean_expression = mean(expression), sd_expression = sd(expression)) %>%
  ungroup()
head(casp_df)

In [ ]:
casp_plot <- (
    ggplot(casp_df, aes(x = Description, y = mean_expression, fill = tissue))
    + geom_bar(stat = "identity", position = "dodge")
    + facet_wrap(~tissue, scales = "free_y")
    # rotate the x-axis labels
    + theme(axis.text.x = element_text(angle = 90, hjust = 1))
    # hide the legend
    + theme(legend.position = "none")
)
casp_plot